In [1]:
%matplotlib inline

In [2]:
#Imports
import numpy as np
import pandas as pd

# Heart Disease Analysis and Prediction - Draft

### 1. Data Acquisition

In [3]:
#TODO use the other dataset - processed.cleveland.data -> rename it and clean it much much carefully
heart_data = pd.read_csv("data/heart-disease.data", header = None)
heart_data.info() # Remove this later

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       303 non-null    float64
 1   1       303 non-null    float64
 2   2       303 non-null    float64
 3   3       303 non-null    float64
 4   4       303 non-null    float64
 5   5       303 non-null    float64
 6   6       303 non-null    float64
 7   7       303 non-null    float64
 8   8       303 non-null    float64
 9   9       303 non-null    float64
 10  10      303 non-null    float64
 11  11      303 non-null    object 
 12  12      303 non-null    object 
 13  13      303 non-null    int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 33.3+ KB


In [4]:
# Shape - describe how many observations and variable do we have
heart_data.shape

(303, 14)

In [5]:
# Display several observation in order to get a sence of the data -> Tell what you see in one, two sentances
heart_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [6]:
heart_data.tail(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1
302,38.0,1.0,3.0,138.0,175.0,0.0,0.0,173.0,0.0,0.0,1.0,?,3.0,0


In [7]:
heart_data.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
10,57.0,1.0,4.0,140.0,192.0,0.0,0.0,148.0,0.0,0.4,2.0,0.0,6.0,0
238,49.0,0.0,2.0,134.0,271.0,0.0,0.0,162.0,0.0,0.0,2.0,0.0,3.0,0
121,63.0,0.0,4.0,150.0,407.0,0.0,2.0,154.0,0.0,4.0,2.0,3.0,7.0,4
9,53.0,1.0,4.0,140.0,203.0,1.0,2.0,155.0,1.0,3.1,3.0,0.0,7.0,1
282,55.0,0.0,4.0,128.0,205.0,0.0,1.0,130.0,1.0,2.0,2.0,1.0,7.0,3


In [8]:
# Columns -> Rename columns and after that describe the variables what they mean -> see the documentation and the research info too
#TODO - rename some columns
heart_data.columns = [
    "age", "sex", "chest_pain_type", "rest_blood_pressure", "cholesterol", "diabetes", "ecg_rest", "heart_rate_max",
    "exercise_angina", "oldpeak", "ST_slope", "vessels", "blood_flow", "heart_disease"]

In [9]:
# Data types -> Describe the data types of the variables but tell also that there are many categorical variables
# Check ranges and value validity of each variable step by step clean -> Cleaning
heart_data.dtypes

age                    float64
sex                    float64
chest_pain_type        float64
rest_blood_pressure    float64
cholesterol            float64
diabetes               float64
ecg_rest               float64
heart_rate_max         float64
exercise_angina        float64
oldpeak                float64
ST_slope               float64
vessels                 object
blood_flow              object
heart_disease            int64
dtype: object

In [10]:
# Clean continous varables
# Age
heart_data["age"] = heart_data["age"].astype(np.int64)
heart_data["age"].min(), heart_data["age"].max() # -> It looks valid

(29, 77)

In [11]:
# Rest blood pressure cleaning -> Continious var
heart_data["rest_blood_pressure"] = heart_data["rest_blood_pressure"].astype(np.int64)
heart_data["rest_blood_pressure"].min(), heart_data["rest_blood_pressure"].max()

(94, 200)

In [12]:
heart_data["cholesterol"] = heart_data["cholesterol"].astype(np.int64)
heart_data["cholesterol"].min(), heart_data["cholesterol"].max()   #Interpret the range - research normal, high and low values

(126, 564)

In [13]:
heart_data["heart_rate_max"] = heart_data["heart_rate_max"].astype(np.int64)
heart_data["heart_rate_max"].min(), heart_data["heart_rate_max"].max() # Ok

(71, 202)

In [14]:
#TODO -> clena oldpeak - research it too

In [15]:

#Clean vessels
heart_data["vessels"].value_counts() # Replace with 0 they are the most

0.0    176
1.0     65
2.0     38
3.0     20
?        4
Name: vessels, dtype: int64

In [16]:
heart_data["vessels"].replace("?", "0.0", inplace = True)
heart_data["vessels"] = pd.to_numeric(heart_data["vessels"]).astype(np.int64)

In [17]:
# Clean categorical vars

# Sex
heart_data["sex"].unique()  # Interpret the values

array([1., 0.])

In [18]:
# Transform sex to categorical variable
def convert_sex_to_text(sex):
    return "Male" if sex == 1 else "Female"

sex_clean = heart_data["sex"].astype(np.int32)
sex_clean = sex_clean.apply(convert_sex_to_text)
heart_data["sex"] = pd.Categorical(sex_clean)

In [19]:
# Chest pain type cleaning  -> Four values -> category
heart_data["chest_pain_type"].unique()  # Interpret values

array([1., 4., 3., 2.])

In [20]:
def convert_chest_pain_type_to_text(pain_type):
    chest_pain_types = {
        1: "typical angina",
        2: "atypical angina",
        3: "non-anginal pain",
        4: "asymptomatic"
    }
    return chest_pain_types.get(pain_type)

chest_pain_clean = heart_data["chest_pain_type"].astype(np.int32)
chest_pain_clean = chest_pain_clean.apply(convert_chest_pain_type_to_text)
heart_data["chest_pain_type"] = pd.Categorical(chest_pain_clean)

In [21]:
# TODO -> clean resting ECG
heart_data["ecg_rest"].unique()

array([2., 0., 1.])

In [22]:
def convert_ecg_rest_to_text(ecg_rest):
    ecg_results = {
        0: "normal",
        1: "ST-T wave abnormality",
        2: "left ventricular hypertrophy"
    }
    return ecg_results.get(ecg_rest)

ecg_clean = heart_data["ecg_rest"].astype(np.int32)
ecg_clean = ecg_clean.apply(convert_ecg_rest_to_text)
heart_data["ecg_rest"] = pd.Categorical(ecg_clean)

In [23]:
heart_data["ST_slope"].unique() # Interpret the values -> Show image

array([3., 2., 1.])

In [24]:
def transform_ST_slope_to_text(ST_slope):
    slope_types = {
        1: "upsloping",
        2: "flat",
        3: "downsloping"
    }
    return slope_types.get(ST_slope)

st_slope_clean = heart_data["ST_slope"].astype(np.int32)
st_slope_clean = st_slope_clean.apply(transform_ST_slope_to_text)
heart_data["ST_slope"] = pd.Categorical(st_slope_clean)

In [25]:
heart_data["blood_flow"].unique()

array(['6.0', '3.0', '7.0', '?'], dtype=object)

In [26]:
heart_data.loc[heart_data["blood_flow"] == "?"]

,age,sex,chest_pain_type,rest_blood_pressure,cholesterol,diabetes,ecg_rest,heart_rate_max,exercise_angina,oldpeak,ST_slope,vessels,blood_flow,heart_disease
87,53,Female,non-anginal pain,128,216,0.0,left ventricular hypertrophy,115,0.0,0.0,upsloping,0,?,0
266,52,Male,asymptomatic,128,204,1.0,normal,156,1.0,1.0,flat,0,?,2


In [27]:
heart_data["blood_flow"].value_counts()

3.0    166
7.0    117
6.0     18
?        2
Name: blood_flow, dtype: int64

In [28]:
heart_data.loc[heart_data["heart_disease"] == 0]["blood_flow"].value_counts()

3.0    129
7.0     28
6.0      6
?        1
Name: blood_flow, dtype: int64

In [29]:
heart_data.loc[heart_data["heart_disease"] == 2]["blood_flow"].value_counts()

7.0    22
3.0     7
6.0     6
?       1
Name: blood_flow, dtype: int64

In [30]:
heart_data.loc[(heart_data["blood_flow"] == "?") & (heart_data["heart_disease"] == 0), "blood_flow"] = "3.0"
heart_data.loc[(heart_data["blood_flow"] == "?") & (heart_data["heart_disease"] == 2), "blood_flow"] = "7.0"

In [31]:
def convert_blood_flow_to_text(blood_flow):
    blood_flow_states = {
        3: "normal",
        6: "fixed defect",
        7: "reversible defect"
    }
    return blood_flow_states.get(blood_flow)
    
blood_flow_clean = pd.to_numeric(heart_data["blood_flow"]).astype(np.int32)
blood_flow_clean = blood_flow_clean.apply(convert_blood_flow_to_text)
heart_data["blood_flow"] = pd.Categorical(blood_flow_clean)

In [32]:
# Clean boolean variables

heart_data["diabetes"].unique()

array([1., 0.])

In [33]:
heart_data["diabetes"] = heart_data["diabetes"].astype(np.bool)

In [34]:
heart_data["exercise_angina"].unique()

array([0., 1.])

In [35]:
heart_data["exercise_angina"] = heart_data["exercise_angina"].astype(np.bool)

In [36]:
# Clean the target
# It is integer valued from 0 (no presence) to 4. Experiments with the Cleveland database have concentrated on simply
#      attempting to distinguish presence (values 1,2,3,4) from absence (value 0). 


# diagnosis of heart disease (angiographic disease status)
# -- Value 0: < 50% diameter narrowing
# -- Value 1: > 50% diameter narrowing
# presence (1-4) absence - 0

heart_data["heart_disease"].unique()

array([0, 2, 1, 3, 4], dtype=int64)

In [37]:
def check_heart_disease_presence(heart_disease_rate):
    return (False if heart_disease_rate == 0 else True)

heart_data["heart_disease"] = heart_data["heart_disease"].apply(check_heart_disease_presence)

In [38]:
heart_data.info() # Compare with the initial result on reading

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   age                  303 non-null    int64   
 1   sex                  303 non-null    category
 2   chest_pain_type      303 non-null    category
 3   rest_blood_pressure  303 non-null    int64   
 4   cholesterol          303 non-null    int64   
 5   diabetes             303 non-null    bool    
 6   ecg_rest             303 non-null    category
 7   heart_rate_max       303 non-null    int64   
 8   exercise_angina      303 non-null    bool    
 9   oldpeak              303 non-null    float64 
 10  ST_slope             303 non-null    category
 11  vessels              303 non-null    int64   
 12  blood_flow           303 non-null    category
 13  heart_disease        303 non-null    bool    
dtypes: bool(3), category(5), float64(1), int64(5)
memory usage: 17.3 KB


In [39]:
heart_data

,age,sex,chest_pain_type,rest_blood_pressure,cholesterol,diabetes,ecg_rest,heart_rate_max,exercise_angina,oldpeak,ST_slope,vessels,blood_flow,heart_disease
0,63,Male,typical angina,145,233,True,left ventricular hypertrophy,150,False,2.3,downsloping,0,fixed defect,False
1,67,Male,asymptomatic,160,286,False,left ventricular hypertrophy,108,True,1.5,flat,3,normal,True
2,67,Male,asymptomatic,120,229,False,left ventricular hypertrophy,129,True,2.6,flat,2,reversible defect,True
3,37,Male,non-anginal pain,130,250,False,normal,187,False,3.5,downsloping,0,normal,False
4,41,Female,atypical angina,130,204,False,left ventricular hypertrophy,172,False,1.4,upsloping,0,normal,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,Male,typical angina,110,264,False,normal,132,False,1.2,flat,0,reversible defect,True
299,68,Male,asymptomatic,144,193,True,normal,141,False,3.4,flat,2,reversible defect,True
300,57,Male,asymptomatic,130,131,False,normal,115,True,1.2,flat,1,reversible defect,True
301,57,Female,atypical angina,130,236,False,left ventricular hypertrophy,174,False,0.0,flat,1,normal,True
